# This file is to take the .xlsx file created by reach_Curator_py38_v3.py script, and adds in the missing rows for T6000, T5000 values
- has multiple steps
1. Backs up the raw .xlsx file created by Reach_Curator_py38_v3.py
2. Takes raw .xlsx file created by Reach_Curator_py38_v3.py --> Adds in missing rows by reading _events.txt file and comparing it --> puts this modifed version with added rows in the root folder Grant_curate
3. Once you have manually curated the session (ie. added reaches) using the Reach_Curator_py38_v3.py
4. this will back up that manually curated .xlsx file into this folder, which it creates if it does not exist already --> completed_manual_curation_backup
5. finally, it will remove any rows that have no values for ReachInit > ReachMax > ReachEnd_ --> it will then put this final cleaned version back into the root folder that the Reach_Curator_py38_v3.py saves and reads from --> Grant_curate

In [746]:
import pandas as pd
import numpy as np
import os
import shutil

## Modify these three variables as needed

In [747]:
root_path = r'G:\Grant\behavior_data\DLC_net' # change this to your root path where all the analysis folder live, i currently use is G:\Grant\behavior_data\DLC_net
root_folder = 'grant_reach10_swingDoor-christie-2025-04-21' # change this to your root folder for one specfic mouse (this should contain multiple sessions)

# Set the specific mouse session to analyze
session_info = '20250426_session001'

### Create new session_option for each session
- class that lets you load data for specific mouse sesssion

In [748]:
main_path = rf'{root_path}\{root_folder}'
print(f'main_path: {main_path}')

class select_mouse_session:
    def __init__(self, mouse, session_date, session_ID):
        self.mouse = mouse
        self.session_date = session_date
        self.session_ID = session_ID

    def __repr__(self):
        return f"Mouse: {self.mouse}, Session Date: {self.session_date}, Session ID: {self.session_ID}"
    
    def get_mouse_session_files(self):
        mouse_session = f"{self.mouse}_{self.session_date}_{self.session_ID}"
        xlsx_file = rf'{main_path}\Grant_curate\{self.session_date}_christielab_{self.session_ID}.xlsx'
        txt_file = rf"G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\videos\{self.session_date}\christielab\{self.session_ID}\{self.session_date}_christielab_{self.session_ID}_events_shifted.txt"
        xlsx_orig = rf'G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\xlsx_backups\curator_direct_backup\{self.session_date}_christielab_{self.session_ID}.xlsx'
        
        return xlsx_file, txt_file, xlsx_orig
    
    def get_mouse_info(self):
        mouse_info = {
            'mouse': self.mouse,
            'session_date': self.session_date,
            'session_ID': self.session_ID
        }
        return mouse_info
    



main_path: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21


In [749]:
print(f"⚠️YOU ARE ABOUT TO MODIFY --> {session_info}⚠️")
print('')

if session_info == '20250421_session001':
    # create a new class instance
    class_intsance = select_mouse_session(mouse='reach10', session_date='20250421', session_ID='session001')
    xlsx_file, txt_file, xlsx_orig = class_intsance.get_mouse_session_files()
    mouse_info = class_intsance.get_mouse_info()
    print(f'mouse_info: {mouse_info}')
    print(f'xlsx_file: {xlsx_file}')

if session_info == '20250424_session001':
    # create a new class instance
    class_intsance = select_mouse_session(mouse='reach10', session_date='20250424', session_ID='session001')
    xlsx_file, txt_file, xlsx_orig = class_intsance.get_mouse_session_files()
    mouse_info = class_intsance.get_mouse_info()
    print(f'mouse_info: {mouse_info}')
    print(f'xlsx_file: {xlsx_file}')
    
if session_info == '20250426_session001':
    # create a new class instance
    class_intsance = select_mouse_session(mouse='reach10', session_date='20250426', session_ID='session001')
    xlsx_file, txt_file, xlsx_orig = class_intsance.get_mouse_session_files()
    mouse_info = class_intsance.get_mouse_info()
    print(f'mouse_info: {mouse_info}')
    print(f'xlsx_file: {xlsx_file}')



⚠️YOU ARE ABOUT TO MODIFY --> 20250426_session001⚠️

mouse_info: {'mouse': 'reach10', 'session_date': '20250426', 'session_ID': 'session001'}
xlsx_file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250426_christielab_session001.xlsx


In [750]:
session_date = mouse_info['session_date']
session_ID = mouse_info['session_ID']
mouse = mouse_info['mouse']
session_save_name = f"{mouse}_{session_date}_{session_ID}"
print('=========================')
print(f'session_save_name: {session_save_name}')
print('')
print(f'mouse: {mouse}')
print(f'session_date: {session_date}')
print(f'session_ID: {session_ID}')


    # ---- Step 1: Make a backup in curator_backup folder
original_dir = os.path.dirname(xlsx_file)
backup_dir = os.path.join(original_dir, 'xlsx_backups',session_save_name,'reach_curator_direct_backup')
os.makedirs(backup_dir, exist_ok=True)
print(f'original_dir: {original_dir}')
print(f'backup_dir: {backup_dir}')


session_save_name: reach10_20250426_session001

mouse: reach10
session_date: 20250426
session_ID: session001
original_dir: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate
backup_dir: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\xlsx_backups\reach10_20250426_session001\reach_curator_direct_backup


## Verify existence of session files

In [751]:
if not os.path.exists(txt_file):
    print(f"⚠️Warning: {txt_file} does not exist⚠️")

backup_file_path = os.path.join(backup_dir, os.path.basename(xlsx_file))
if os.path.exists(backup_file_path):
    print(f"⚠️Warning: Backup file already exists at {backup_file_path}⚠️")
else:
        # Check if files exist
    if not os.path.exists(xlsx_file):
        print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
    else:
        # Copy the original file to the backup directory
        shutil.copy2(xlsx_file, backup_file_path)
        print(f"✅ Backup saved to: {backup_file_path}")

# ---- (your parsing & merging code goes here)
# ... your code to build `combined` DataFrame ...

print(f'backup_dir: {backup_file_path}')
print('=========================')
print(f'xlsx_file: {xlsx_file}')
print('=========================')
print(f'txt_file: {txt_file}')
print('=========================')
print(f'xlsx_orig: {xlsx_orig}')


✅ Backup saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\xlsx_backups\reach10_20250426_session001\reach_curator_direct_backup\20250426_christielab_session001.xlsx
backup_dir: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\xlsx_backups\reach10_20250426_session001\reach_curator_direct_backup\20250426_christielab_session001.xlsx
xlsx_file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250426_christielab_session001.xlsx
txt_file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\videos\20250426\christielab\session001\20250426_christielab_session001_events_shifted.txt
xlsx_orig: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\xlsx_backups\curator_direct_backup\20250426_christielab_session001.xlsx


### This is the cell that takes the .xlsx file and the .txt file and creates a new .xlsx file with the manual additions
- makes a new folder called manual_additions if it does not exist
- reads the .xlsx file into a pandas dataframe
- reads the .txt file into a pandas dataframe
- creates a new dataframe with the manual additions
- saves the new dataframe to a new .xlsx file



In [752]:
# Load existing curated xlsx
print(f"Loading existing xlsx file: {xlsx_file}")
df_orig = pd.read_excel(xlsx_file)
df_orig.head()

Loading existing xlsx file: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250426_christielab_session001.xlsx


,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected,ReachType
0,1816,2100,2185,2202,2205,0,missed,2098,1869,short
1,4229,4661,4664,4689,4701,0,dropped,4659,4283,short
2,6725,6858,6873,6889,6918,0,consumed_first,6856,6779,on_pellet
3,8911,9414,9428,9448,9459,0,missed,9412,8999,short
4,12129,12336,12353,12374,12385,0,missed,12333,12182,short


In [754]:

class modify_curator_xlsx_file(object):
    def __init__(self, xlsx_file, txt_file,save_df=True):
        self.txt_file = txt_file
        self.xlsx_file = xlsx_file
        self.save_df = save_df

        
    def create_new_xlxs_file(self):
        save_df = self.save_df
        # Load the existing xlsx file
        df_orig = pd.read_excel(self.xlsx_file)
        # Check if the file is empty
        txt_file = self.txt_file
        if not os.path.exists(txt_file):
            print(f"⚠️Warning: {txt_file} does not exist⚠️")
            return None
        # Check if the xlsx file is empty
        xlsx_file = self.xlsx_file
        if not os.path.exists(xlsx_file):
            print(f"⚠️Warning: {xlsx_file} does not exist⚠️")
            return None

        if df_orig.empty:
            print("⚠️Warning: The xlsx file is empty⚠️")
            return None
        
        # Normalize column names just in case
        df_orig.columns = df_orig.columns.str.strip()

        # Load and parse txt as done earlier
        events = []
        current_event = {}
        with open(txt_file, 'r') as f:
            for line in f:
                if line.strip():
                    key, value = line.strip().split('\t')
                    value = int(value)
                    if key == 'T6000_played':
                        if current_event:
                            events.append(current_event)
                        current_event = {'T6000': value}
                    elif key == 'pellet_detected':
                        current_event['pellet_detected'] = value
                    elif key == 'pellet_delivery':
                        current_event['pellet_delivery'] = value
                    elif key == 'T5000_played':
                        current_event['T5000'] = value
        if current_event:
            events.append(current_event)

        events_df = pd.DataFrame(events)

        # Find which T6000s are missing from original file
        existing_T6000s = df_orig['T6000'].values
        new_rows = events_df[~events_df['T6000'].isin(existing_T6000s)].copy()

        # Fill in missing columns to match structure
        new_rows['reachInit'] = np.nan
        new_rows['reachMax'] = np.nan
        new_rows['reachEnd'] = np.nan
        new_rows['stim'] = 0
        new_rows['behaviors'] = np.nan

        # # Reorder columns to match original file
        # new_rows = new_rows[['reachinit', 'reachmax', 'reachend', 'stim',
        #                      'pellet_delivery', 'pellet_detected', 't6000', 't5000', 'behaviors']]

        # Reorder columns to match original file
        new_rows = new_rows[['T6000', 'T5000', 'reachInit', 'reachMax',
                            'reachEnd', 'stim','behaviors' ,'pellet_delivery', 'pellet_detected']]

        # Combine and sort by T6000
        combined = pd.concat([df_orig, new_rows], ignore_index=True)
        combined = combined.sort_values(by='T6000').reset_index(drop=True)

        # Force final column order
        combined = combined[['T6000', 'T5000', 'reachInit', 'reachMax',
                            'reachEnd', 'stim', 'behaviors',
                            'pellet_delivery', 'pellet_detected']]


        # ---- Step 2: Save output as the same original file (overwrite it)
        xlsx_save_path = xlsx_file
        self.xlsx_file = xlsx_save_path

        original_dir = os.path.dirname(self.xlsx_file)
        backup_dir = os.path.join(original_dir,'xlsx_backups' ,'reach_curator_direct_backup')
        backup_file_path = os.path.join(backup_dir, os.path.basename(self.xlsx_file))

        backup_added_rows = os.path.join(original_dir, 'xlsx_backups' ,session_save_name,'added_missing_rows_backup')
        os.makedirs(backup_added_rows, exist_ok=True)
        backup_added_rows_file_path = os.path.join(backup_added_rows, os.path.basename(self.xlsx_file))

        # Save the combined DataFrame to the same xlsx file
        if save_df:
            if os.path.exists(backup_file_path):
                combined.to_excel(xlsx_save_path, index=False)
                combined.to_excel(backup_added_rows_file_path, index=False)
                print(f"✅ Final merged Excel saved to: {xlsx_save_path}")
                print(f"✅ Backup saved to: {backup_added_rows_file_path}")
                return combined, xlsx_save_path
            else:
                print(f"⚠️Warning: Backup file doesnt exists⚠️")
                print('❌ Stopping the script to avoid overwriting the xlsx file without a back up❌')
                print(f"⚠️Warning: {backup_file_path} does not exist⚠️")
                return None, None
        else:
            print(f"⚠️Warning: DataFrame not saved to {xlsx_save_path}⚠️")
            print('set save_df to True to save the DataFrame')
            return combined
    

    def drop_empty_reach_rows(self):
      
        df = pd.read_excel(self.xlsx_file)
        # back up the newly created xlsx file
        backup_dir_02 = os.path.join(os.path.dirname(self.xlsx_file),'xlsx_backups', session_save_name,'completed_manual_curation_backup')
        os.makedirs(backup_dir_02, exist_ok=True)
        backup_file_path_02 = os.path.join(backup_dir_02, os.path.basename(self.xlsx_file))
        if os.path.exists(backup_file_path_02):
            print(f"⚠️Warning: Backup file already exists at {backup_file_path_02}⚠️")
            print('❌ Stopping the script to avoid overwriting the backup file ❌')
            return None 
        else:
            # Copy the original file to the backup directory
            shutil.copy2(self.xlsx_file, backup_file_path_02)
            print(f"✅ Backup saved to: {backup_file_path_02}")
        """
        Drop specified rows from the DataFrame.
        """
        # Drop bad rows
        df = df.dropna(subset=['reachInit']).reset_index(drop=True)
         # save the DataFrame to the same xlsx file
        if self.save_df:
            df.to_excel(self.xlsx_file, index=False)
            print(f"✅ Final merged Excel saved to: {self.xlsx_file}")   
              
        return df
    
    def load_orig_xlsx_file(self):
        """
        Load the original xlsx file.
        """
        original_dir = os.path.dirname(self.xlsx_file)
        backup_dir = os.path.join(original_dir, 'xlsx_backups',session_save_name ,'reach_curator_direct_backup')
        backup_file_path = os.path.join(backup_dir, os.path.basename(self.xlsx_file))

        df_orig = pd.read_excel(backup_file_path)
        return df_orig, backup_file_path
    
    def load_added_rows_xlsx(self):
        """
        Load the added rows xlsx file.
        """
        original_dir = os.path.dirname(self.xlsx_file)
        backup_added_rows = os.path.join(original_dir,'xlsx_backups' ,session_save_name, 'added_missing_rows_backup')
        backup_added_rows_file_path = os.path.join(backup_added_rows, os.path.basename(self.xlsx_file))

        df_added_rows = pd.read_excel(backup_added_rows_file_path)
        return df_added_rows, backup_added_rows_file_path
    
    
    def load_final_xlsx(self):
        """
        Load the combined xlsx file.
        """
        df_combined = pd.read_excel(self.xlsx_file)
        return df_combined, self.xlsx_file




In [755]:
safe_fail_modify = True

In [756]:
if safe_fail_modify:
    mouse_sessions = modify_curator_xlsx_file(xlsx_file, txt_file,save_df=True)
    df_modied, xlsx_saved_file = mouse_sessions.create_new_xlxs_file()
    df_modied
else:
    print(f"⚠️Warning: safe_fail actviated⚠️")
    print('if you want to run the code, please re-run this cell')
    print('Note this ')
    safe_fail_modify = True

✅ Final merged Excel saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\20250426_christielab_session001.xlsx
✅ Backup saved to: G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\xlsx_backups\reach10_20250426_session001\added_missing_rows_backup\20250426_christielab_session001.xlsx


In [757]:
# Load existing curated xlsx
df_added_missing_rows = pd.read_excel(xlsx_saved_file)
df_added_missing_rows.head(5)

,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected
0,1816,2100.0,2185.0,2202.0,2205.0,0,missed,2098.0,1869.0
1,4229,4661.0,4664.0,4689.0,4701.0,0,dropped,4659.0,4283.0
2,6725,6858.0,6873.0,6889.0,6918.0,0,consumed_first,6856.0,6779.0
3,8911,9414.0,9428.0,9448.0,9459.0,0,missed,9412.0,8999.0
4,12129,12336.0,12353.0,12374.0,12385.0,0,missed,12333.0,12182.0


In [758]:
df_orig, backup_file_path = mouse_sessions.load_orig_xlsx_file()
df_added_missing_rows, backup_added_rows_file_path = mouse_sessions.load_added_rows_xlsx()

In [759]:
orig_reach_count = df_orig.shape[0]
added_row_count = df_added_missing_rows.shape[0]
print(f'loading orginal_df for reach events from --> {backup_file_path}')
print(f'loading df with missing rows added from--> {backup_added_rows_file_path}')
# Count the number of reach events
print('')
print(f"Original reach events: {len(df_orig)}")
print(f"Added reach events: {len(df_added_missing_rows)}")
print('')
print(f'❌ Reaches Missed: {added_row_count- orig_reach_count}')


loading orginal_df for reach events from --> G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\xlsx_backups\reach10_20250426_session001\reach_curator_direct_backup\20250426_christielab_session001.xlsx
loading df with missing rows added from--> G:\Grant\behavior_data\DLC_net\grant_reach10_swingDoor-christie-2025-04-21\Grant_curate\xlsx_backups\reach10_20250426_session001\added_missing_rows_backup\20250426_christielab_session001.xlsx

Original reach events: 52
Added reach events: 57

❌ Reaches Missed: 5


In [760]:
df_orig.head(15)

,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected,ReachType
0,1816,2100,2185,2202,2205,0,missed,2098,1869,short
1,4229,4661,4664,4689,4701,0,dropped,4659,4283,short
2,6725,6858,6873,6889,6918,0,consumed_first,6856,6779,on_pellet
3,8911,9414,9428,9448,9459,0,missed,9412,8999,short
4,12129,12336,12353,12374,12385,0,missed,12333,12182,short
5,14405,14795,14810,14833,14855,0,consumed_first,14793,14492,above
6,16892,17207,17221,17240,17245,0,dropped,17205,16978,short
7,19305,19441,19451,19464,19472,0,missed,19439,19362,short
8,23610,24047,24060,24080,24086,0,dropped,24044,23664,above
9,28216,28351,28367,28381,28384,0,dropped,28349,28273,above


In [778]:
df_added_missing_rows.head(15)

,T6000,T5000,reachInit,reachMax,reachEnd,stim,behaviors,pellet_delivery,pellet_detected
0,1816,2100.0,2185.0,2202.0,2205.0,0,missed,2098.0,1869.0
1,4229,4661.0,4664.0,4689.0,4701.0,0,dropped,4659.0,4283.0
2,6725,6858.0,6873.0,6889.0,6918.0,0,consumed_first,6856.0,6779.0
3,8911,9414.0,9428.0,9448.0,9459.0,0,missed,9412.0,8999.0
4,12129,12336.0,12353.0,12374.0,12385.0,0,missed,12333.0,12182.0
5,14405,14795.0,14810.0,14833.0,14855.0,0,consumed_first,14793.0,14492.0
6,16892,17207.0,17221.0,17240.0,17245.0,0,dropped,17205.0,16978.0
7,19305,19441.0,19451.0,19464.0,19472.0,0,missed,19439.0,19362.0
8,23610,24047.0,24060.0,24080.0,24086.0,0,dropped,24044.0,23664.0
9,28216,28351.0,28367.0,28381.0,28384.0,0,dropped,28349.0,28273.0
